In [ ]:
from  rrfpy.rrfpy_pandas import *
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as ps
import plotly.graph_objects as go
from IPython.display import display, Latex

## $\tau$ Decays

We start with considering very simple decay: $\tau^- \to e^- \bar{\nu}_e \nu_\tau$

The file we are analyzinf was created in the **build/** directory using the command

    ./simpleEvtGenRO.exe tau- ../src/tau_enu.dec  100000
    cp evtOutput.root evtOutput_tau.root
    
Let is download it first

In [ ]:
rrF = rrFpy_pandas("../build/evtOutput_tau.root")

We can see that 100000 events were loaded

In [ ]:
rrF.size()

and all these events correspond to shown above $\tau$-lepton decay

In [ ]:
rrF["reac"].value_counts()

The distributions over various kinematical vars can be extracted using **["var"]** notation. Here is the energy of the first produced particle (i.e. electron), for example

In [ ]:
distE = rrF["E_1"]
distE

The result is usual **pandas.Series** object, so you can do whatever you want with is.

Calculate the mean:

In [ ]:
distE.mean()

or even plot the histogram

In [ ]:
distE.hist(bins=50, density = True)
mtau = rrF["m_0"].mean()
e = np.linspace(0, mtau/2, 100)
plt.plot(e, 16/mtau**3*(3-4*e/mtau)*e**2, 'r', linewidth = 5);
plt.xlabel(r"$E_e,\,\mathrm{GeV}$")
plt.ylabel(r"$d\mathrm{Br}/dE_e,\,\mathrm{GeV}^{-1}$")
plt.title(rrF["reac"].iloc[0])
plt.show()

Note that the result prefectly agrees with the textbook variant
$$\frac{d\mathrm{Br}}{dE}= \frac{16}{m_\tau^3}\left[3-\frac{4E}{m_\tau}\right]E^2,$$
that is shown with red line in the above plot

All other distributions can also be easily constricted. Here, for example, are distributions over different square masses

In [ ]:
rrF[["m2_12", "m2_13", "m2_23"]].plot.hist(bins=50, histtype = 'step', density = True)
plt.xlabel(r"$m^2,\,\mathrm{GeV^2}$")
plt.ylabel(r"$d\mathrm{Br}/dm^2,\,\mathrm{GeV}^{-2}$")
plt.title(rrF["reac"].iloc[0])
plt.show()
plt.show()

As expected, the relation
$$s+t+u = \sum_i m_i^2$$
does hold for all events

In [ ]:
np.sqrt(rrF["m2_12"]+rrF["m2_13"] + rrF["m2_23"])

Any cuts can easily be imposed to the data set

In [ ]:
rrFcut = rrF.cut("E_1>0.4")
print("%d events survived after the cut (%d %%)" % (rrFcut.size(), 100*rrFcut.size()/rrF.size()) )
rrFcut[["m2_12", "m2_13", "m2_23"]].plot.hist(bins=50, alpha = 0.5, density = True)
plt.show()

## $B_c$ Decays

Let us try the package on more serious example: $B_c^+$ meson decays.

The file we will be using here is created in **../build/** directory by the commands

    ./simpleEvtGenRO.exe B_c+ ../src/Bc.dec  100000
    cp evtOutput.root evtOutput_Bc.root
    
As you can see from the decay file

    noPhotos
    Decay B_c+
    1. J/psi pi+ pi+ pi- BC_VHAD 1;
    1. J/psi pi+ pi+ pi- pi- pi+ BC_VHAD 1;
    1. J/psi K+ K- pi+ BC_VHAD 1;
    1. J/psi K+ pi+ pi- BC_VHAD 1;
    Enddecay
    End
    
the whole bunch of reactions is stored in the resulting ROOT

Let us first load it

In [ ]:
rrF = rrFpy_pandas()
rrF.load_ROOT("../build/evtOutput_Bc.root")

Here is the list of all reactions (with number of decays for each of them)

In [ ]:
reacs = rrF["reac"].value_counts()
print(reacs)

This data can be represented in nice graphical way

In [ ]:
reacs.plot.barh()
plt.xlabel("# of records")
plt.ylabel("reaction")
plt.show()


First we will discuss the $B_c^+ \to J/\psi \pi^+ \pi^+ \pi^-$ decay. To select the corresponding records we can use the **cut()** method (note that PDG codes for $\pi^+$ and $\pi^-$ particles are 211 and -211 respectively)

In [ ]:
rrf_3pi = rrF.cut(["ntr=5", "id_2=211","id_3=211","id_4=-211"])
rrf_3pi["reac"].value_counts()

As you can see, only the reaction we are interested in is left.

The same result can be obtained with **filter** command, that accepts a boolean list as an argument (see the similar mask functionality of the **pandas.DataFrame** objects)

In [ ]:
rrf_3pi  = rrF.filter( (rrF["ntr"]==5) & (rrF["name_2"] == "pi+") & (rrF["name_3"]=="pi+") & (rrF["name_4"]=="pi-") )
rrf_3pi["reac"].value_counts()

Sometimes the last notation is not very convenient, but it is much more powerfull then the **cut()** method

In any case we can extract from the resulting object the distribution over any kinematic variable using [] notation

In [ ]:
rrf_3pi["m_23"]

The result is **pandas.Series** object, so you can do whatever you want with it.

Calculate the mean, for example

In [ ]:
rrf_3pi["m_23"].mean()

Or plot it

In [ ]:
rrf_3pi["m2_23"].hist(bins=30)
plt.xlabel(r"$m_{23}$")
plt.show()

You can use the whole python functionality, so very nice plots can be easily created in small time

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1)
rrf_3pi = rrF.cut(["ntr=5","id_2=211"])
variables = {'m_23':r'$\pi^+\pi^+$', 'm_24':r'$\pi^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
for var, name in variables.items():
    rrf_3pi[var].hist(label=name, alpha=0.5, bins=50)
fig.suptitle(rrf_3pi["reac"].drop_duplicates().iloc[0])
ax.axvline(x=0.77, linestyle = '--', color = 'r')
plt.legend()
plt.show()

In the above plot you can easily see that distributions over two $\pi^+\pi^-$ masses are the same (two $\pi^+$ mesons are identical) and show the peak at $m_{\pi\pi}\sim m_\rho \approx 770$ MeV (is shown with the red dashed line).

In the case of $m_{\pi^+\pi^+}$ distribution such a peak is absent.

The plot can be made more beautiful and interactinve with the **pyplot** package. Now you can use your mouse cursor to see the exact position of the peak (pay attention to the hover text)

In [ ]:
rrf_3pi = rrF.cut(["ntr=5","id_2=211"])
data = rrf_3pi
variables = {'m_23':r'$\pi^+\pi^+$', 'm_24':r'$\pi^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=data[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = data["reac"].drop_duplicates().iloc[0])
fig.update_traces(opacity=0.5)
fig.show()

With minimal modufications we draw the similar distributions for other reactions

In [ ]:
rrf_Kpipi = rrF.cut(["ntr=5","id_2=321","id_3=211"])
variables = {'m_23':r'$K^+\pi^+$', 'm_24':r'$K^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=rrf_Kpipi[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = rrf_Kpipi["reac"].drop_duplicates().iloc[0])
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
rrf_KKpi = rrF.cut(["ntr=5","id_2=321","id_3=-321"])
variables = {'m_23':r'$K^+K^-$', 'm_24':r'$K^+\pi^+$', 'm_34':r'$K^-\pi^+$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=rrf_KKpi[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = rrf_KKpi["reac"].drop_duplicates().iloc[0])
fig.update_traces(opacity=0.5)
fig.show()